# importing Packages

In [18]:
#The required python packages are as follows
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle

# Loading and extracting Data

In [19]:
#Now we load the json file and extract the required data
with open('Bank_FAQs.json') as file:
    data = json.load(file)

In [20]:
#The variable "training_sentences" holds all the training data (which are the sample messages in each intent category) and the "training_lables" variable holds all the target lables correspond to each training data.
training_sentences = []
training_labels = []
labels = []
responses = []


In [21]:
#We will define few simple intents and bunch of messages that correspondes to thos intents and also map some responses according to each intent category.we will create a JSON file named "BankFAQs.json" including these data as follows.
for intent in data['BankFAQs']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [22]:
num_classes

7

In [23]:
training_sentences

['What do you mean by exclusions under the Accidental Protection Plan - Hospital Cash',
 'How do I make a claim',
 'When does my Revive Policy for Personal Accident start',
 'What plans are available under  the Revive Policy for Personal Accident ',
 'What benefit do I get from the Revive Policy for Personal Accident if I break my bones',
 'Is there an age limit to buy an  Accidental Protection Plan - Hospital Cash policy',
 'Who can be a beneficiary under the Accidental Protection Plan - Hospital Cash',
 'How do I renew my Accidental Protection Plan - Hospital Cash policy',
 'Will I get a renewed Revive Policy for Personal Accident policy',
 'What is the definition of a dependent child',
 'Does  the Accidental Protection Plan - Hospital Cash policy cover natural death and daily sickness as well',
 'Does this the Revive Policy for Personal Accident cover natural death and daily sickness',
 'What benefit do I get from the Accidental Protection Plan - Hospital Cash policy if I break my b

In [24]:
training_labels

['Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insurance',
 'Insu

In [25]:
labels

['Insurance',
 'Security',
 'Accounts',
 'Cards',
 'Fundstransfer',
 'Investments',
 'Loans']

In [26]:
responses

[['Exclusions are situations or conditions where HDFC ERGO is not liable to pay the benefits to the insured person in the event of an accident.'],
 ['You can make a claim by calling our toll-free number 1800-226-226 (MTNL or BSNL) or 1800-2-700-700 (any line) or 022 6638 4800 (local/STD charges apply). We will then assist you with the submission of the documents required and the process will be completed within 7 working days, once all the required documents are submitted.'],
 ['It will commence within 15 days from the date of receipt of the form and premium payment.'],
 ['HDFC ERGO offers a wide range of sums insured ranging from Rs. 2.5 Lakhs to Rs. 15 Lakhs, with four plan options to choose from: Self Plan Self and Family Plan Self + Dependent Parents Add-On Self and Family + Dependent Parents Add-On'],
 ['You get 10% of the sum insured up to Rs. 50,000 (for dependent parents) if you break your bones in an accident.'],
 ['We can insure anyone between the age of 18 and 69 years.'],
 

# PreProcessing

In [27]:
#Then we use "labelencoder()" function provided by scikit-learn to convert the target lables into a model understandable form.
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [28]:
training_labels

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [29]:
#We vectorize our text dta corpus by using the "Tokenizer" class and it alows us to limit our vocabulary size up to some defined number. when we use this class for the text pre-processing task, by default all punctuations will be removed, turning the texts into space-separated sequences of words, and thes sequences are split into lists of token. they will then be indexed or vectorized. 
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=20)#The "pad_sequences" method is used to make all the training text sequences into the same size.

# Model Building

In [30]:
#Let's define our Neural network architecture for the proposed model and for that we use the "Sequential" model class of Keras.
model = Sequential()
model.add(Embedding(5000,16, input_length=20))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            80000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 7)                 119       
                                                                 
Total params: 80,663
Trainable params: 80,663
Non-trainable params: 0
__________________________________________________

# Model Training

In [31]:
# Now we are ready to train our model.Simply we can call the "fit" method with training data and lables.
epochs = 100
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs, batch_size=8)

Epoch 1/100
70/70 [==============================] - 2s 8ms/step - loss: 1.9320 - accuracy: 0.1931
Epoch 2/100
70/70 [==============================] - 1s 7ms/step - loss: 1.8693 - accuracy: 0.2563
Epoch 3/100
70/70 [==============================] - 1s 8ms/step - loss: 1.7091 - accuracy: 0.4513
Epoch 4/100
70/70 [==============================] - 1s 8ms/step - loss: 1.4381 - accuracy: 0.5776
Epoch 5/100
70/70 [==============================] - 1s 8ms/step - loss: 1.1671 - accuracy: 0.5993
Epoch 6/100
70/70 [==============================] - 1s 8ms/step - loss: 0.9575 - accuracy: 0.6986
Epoch 7/100
70/70 [==============================] - 1s 8ms/step - loss: 0.7755 - accuracy: 0.8484
Epoch 8/100
70/70 [==============================] - 1s 8ms/step - loss: 0.6232 - accuracy: 0.8755
Epoch 9/100
70/70 [==============================] - 1s 8ms/step - loss: 0.5013 - accuracy: 0.8953
Epoch 10/100
70/70 [==============================] - 1s 8ms/step - loss: 0.4020 - accuracy: 0.9224
Epoch 11/

70/70 [==============================] - 1s 8ms/step - loss: 5.0320e-04 - accuracy: 1.0000
Epoch 83/100
70/70 [==============================] - 0s 7ms/step - loss: 4.7762e-04 - accuracy: 1.0000
Epoch 84/100
70/70 [==============================] - 0s 7ms/step - loss: 4.5418e-04 - accuracy: 1.0000
Epoch 85/100
70/70 [==============================] - 0s 7ms/step - loss: 4.3546e-04 - accuracy: 1.0000
Epoch 86/100
70/70 [==============================] - 0s 7ms/step - loss: 4.0621e-04 - accuracy: 1.0000
Epoch 87/100
70/70 [==============================] - 0s 7ms/step - loss: 3.9801e-04 - accuracy: 1.0000
Epoch 88/100
70/70 [==============================] - 0s 7ms/step - loss: 3.6622e-04 - accuracy: 1.0000
Epoch 89/100
70/70 [==============================] - 1s 7ms/step - loss: 3.5862e-04 - accuracy: 1.0000
Epoch 90/100
70/70 [==============================] - 1s 7ms/step - loss: 3.3089e-04 - accuracy: 1.0000
Epoch 91/100
70/70 [==============================] - 1s 8ms/step - loss: 3.1

In [32]:
# After training,it is better to save all the required files in order to use it at the inference time. so that we save the trained model,fitted tokenizer object and fitted lable encoder object.

In [33]:
# To save the trained model
model.save("Bank_FAQ_ChatBot")


INFO:tensorflow:Assets written to: Bank_FAQ_ChatBot/assets


In [34]:
#To save the fitted tokenizer 
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
#To save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle


In [36]:
with open("Bank_FAQs.json") as file:
    data = json.load(file)

# Model Testing

In [37]:
def chat():
    # load trained model
    model = keras.models.load_model('Bank_FAQ_ChatBot')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['BankFAQs']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

In [38]:
print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: Is there an age limit to buy an  Accidental Protection Plan - Hospital Cash policy
ChatBot: Exclusions are situations or conditions where HDFC ERGO is not liable to pay the benefits to the insured person in the event of an accident.
ChatBot: You can make a claim by calling our toll-free number 1800-226-226 (MTNL or BSNL) or 1800-2-700-700 (any line) or 022 6638 4800 (local/STD charges apply). We will then assist you with the submission of the documents required and the process will be completed within 7 working days, once all the required documents are submitted.
ChatBot: It will commence within 15 days from the date of receipt of the form and premium payment.
ChatBot: HDFC ERGO offers a wide range of sums insured ranging from Rs. 2.5 Lakhs to Rs. 15 Lakhs, with four plan options to choose from: Self Plan Self and Family Plan Self + Dependent Parents Add-On Self and Family + Dependent Parents Add-On
ChatBot: You get 10% of the sum

ChatBot: The insured shall arrange for submission of the following documents required to process a claim within 45 days from the date of intimation. Duly completed claim form Original discharge summary Consultation note/ relevant treatment papers All relevant medical reports along with supporting invoices and doctors requisition advising the same Original and final hospitalization bills with detailed break-up Pharmacy Bills along with prescriptions Any other documents as may be required by the company On receipt of the claim documents the claim will be processed in accordance with the terms and conditions of the policy. View more
ChatBot: In the case of a claim under the policy, you should immediately intimate us on our helpline numbers.
ChatBot: No pre-policy medical check up is required for individuals up to the age of 45 years.
ChatBot: Critical Illness insurance provides you and your family additional financial security on the diagnosis of a critical illness. The policy provides a 

ChatBot: Organ Donor Expenses are all the hospitalisation expenses incurred by the donor for the donation of an organ (excluding the cost of the organ) to the insured during the course of an organ transplant.
ChatBot: Any condition, ailment, injury of related condition(s) for which the insured person has had signs or symptoms and/or was diagnosed and/or has received medical advice/treatment within 48 months prior to the insured’s first policy with the company.
ChatBot: For hospitalization /claim, customer can contact via Call: 1800 200 1999 / 1860 2000 700 Fax: 1860 2000 600 Email: [email protected] Through IPO: Login to Insurance Portfolio Organizer (IPO) from HDFC Ergo website home page
ChatBot: Yes, you can avail up to Rs. 15,000 as tax benefit under 'Section 80D'. In case you are a senior citizen, you can avail up to Rs. 20,000 as tax benefit under 'Section 80D'. Subject to changes in Tax laws
ChatBot: You can insure yourself for Rs. 3 Lakhs, Rs. 4 Lakhs or Rs. 5 Lakhs. You can cho

ChatBot: In the event that you wish to change the mode of repayment/bank account for your loan repayment, you may visit the nearest the Retail Loan Service Center and submit your request with the revised account mandate/instructions. You can opt to repay through alternate repayment instructions as follows: 1) Standing Instructions: We offer the Standing Instructions (SI) facility whereby we can directly debit your HDFC Bank account for EMI amounts on due dates. Please submit the following at your nearest Retail Loan Service Center: Request form for swap of repayment instructions. 3 Security cheques favoring HDFC Bank Ltd Loan Account No. << >>, each undated and filled with a value equal to the sum of 3 EMIs. SI mandate duly filled and verified by HDFC Bank. In case a bearer submits the request on your behalf, a bearer authorisation along with KYC documents(self attested copy and original for validation) of the bearer,i.e. Identity and Signature Proof like PAN Card, Aadhar Card, Driving

ChatBot: CIBIL stands for Credit Information Bureau India Limited. CIBIL's aim is to fulfil the need of credit granting institutions for comprehensive credit information by collecting, collating and disseminating credit information pertaining to both commercial and consumer borrowers, to a closed user group of members. Banks, Financial Institutions, Non-Banking Financial Companies, Housing Finance Companies and Credit Card Companies use CIBIL's services. Data sharing is based on the Principle of Reciprocity, which means that only Members who have submitted all their credit data may access Credit Information Reports from CIBIL. The establishment of CIBIL is an effort made by the Government of India and the Reserve Bank of India to improve the functionality and stability of the Indian financial system by containing NPAs while improving credit grantors' portfolio quality. CIBIL provides a vital service, which allows its Members to make informed, objective and faster credit decisions. View

ChatBot: You may obtain a duplicate NOC/Form 35 for your vehicle by visiting any of our the Retail Loan Service Center and submit the following documents: Request form for Duplicate NOC/Form 35. Self attested copy of the Registration Certificate book, duly self attested by you, along with the Original Registration Certificate book for validation. Applicable fee of Rs. 500/- vide a cheque/payorder/demand draft favouring “HDFC Bank Ltd Loan Account No. << >>. In case a bearer submits the request on your behalf, a bearer authorisation along with KYC documents(self attested copy and original for validation) of the bearer,i.e. Identity and Signature Proof like PAN Card, Aadhar Card, Driving License or Passport Copy will also need to be provided. Please note that the Loan Closure NOC/Form 35 will not be issued if any of the following conditions apply: Copy of RC Book is not submitted The same vehicle is encumbered to the Bank for any other loan facility Any notice from a competent authority 

ChatBot: Registration Certificate-Important advise HDFC Bank is in no way responsible for the delivery of the Registration Certificate. The Borrower shall submit the certified true copy of the Registration Certificate relevant to the vehicle for which the loan has been taken.The registration certificate has to be submitted within 60 days of having taken delivery of the vehicle or 150 days from the date of disbursal of the loan whichever is earlier.The Borrower is also required to submit copy of the Insurance policy as well as Invoice to the bank,within this period. Vehicle Delivery The role of HDFC Bank is to be a finance provider to the borrower and hence the Bank is not responsible for delay, non-delivery or any defect, damage in quality and workmanship of the Vehicle. The Bank is also not liable or responsible in case of defect or dispute of any nature in the title (even if the vehicle is found to be a stolen Vehicle) or ownership of the Vehicle. Loan cancellation In case of loan ca

ChatBot: You may obtain the NOC for a duplicate Registration Certificate book for your vehicle loan, by visiting any of our Retail Loan Service Center and submitting the following documents: Request form for applying for the required NOC. Self attested copy of the Registration Certificate book, duly self attested by you for validation. Copy of FIR or certificate from police or Acknowledgement from Police on written complaint with General Diary No. Applicable fee of Rs. 500//- vide a cheque/payorder/demand draft favouring HDFC Bank Ltd Loan Account No. << >>. In case a bearer submits the request on your behalf, a bearer authorisation along with KYC documents(self attested copy and original for validation) of the bearer,i.e. Identity and Signature Proof like PAN Card, Aadhar Card, Driving License or Passport Copy will also need to be provided. View more
ChatBot: A service charge of Rs.500 is applicable for changes in loan repayment instructions issued from non-HDFC Bank Account (ECS or P

ChatBot: We offer Pre-owned Car Loans to the following: 1. Salaried individuals in the age group of 21 to 60 years (at the end of the tenure) 2. Self-employed individuals in the age group of 21 to 65 years (at the end of the tenure) 3. Partnership Firms 4. Public & Private Ltd. Companies 5. HUFs and Trusts Learn how to get an HDFC Bank Pre-owned Car Loans.
ChatBot: No, but if your income does not meet our credit criteria then you may be required to have a co-applicant.
ChatBot: To get an HDFC Bank Pre-owned Car Loans, just fill the Online Application Form and our representative will get in touch with you shortly. Or you can also Call us or Visit our branch.
ChatBot: Yes, you can pay your missed EMI online. You need to provide your HDFC Bank Loan details, confirm payment towards HDFC Bank Loan account post which you will receive an online transaction confirmation and a Transaction Reference Number. The link for making HDFC Bank overdue loan repayment online is given below. https://www.b

ChatBot: You may obtain the NOC for fitting an LPG/CNG kit on your registered vehicle under finance with us by visiting any of our the Retail Loan Service Center and submitting the following documents: Request form for LPG/CNG NOC. Self attested copy of the Registration Certificate book, duly self attested by you, along with the Original Registration Certificate book for validation. Applicable fee of Rs. 500/- vide a cheque/payorder/demand draft favouring HDFC Bank Ltd Loan Account No. << >>. In case a bearer submits the request on your behalf, a bearer authorisation along with KYC documents (self attested copy and original for validation) of the bearer, i.e. Identity and Signature Proof like PAN Card, Aadhar Card, Driving License or Passport Copy will also need to be provided. View more
ChatBot: You may obtain a duplicate NOC/Form 35 for your vehicle by visiting any of our the Retail Loan Service Center and submit the following documents: Request form for Duplicate NOC/Form 35. Self a